#### TFT API 에 대해서 설명되어있는 사이트 : https://velog.io/@twa02189/TFT-API-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%84%A4%EB%AA%85

In [1]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

#### 기본 키 저장 + baser URL 저장 + request header 저장

In [2]:
# api_key 저장
api_key = 'RGAPI-ca44b272-60ec-4589-8980-8f28e65ca212' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

#### 챌린저, 마스터, 그랜드 마스터 구간 유저들의 닉네임 추출 

##### LEAGUE/V1 API 에서 랭크 구간별 유저 정보 REQUEST

In [3]:
# challenger

challenger_url = base_url + 'league/v1/challenger'

challenger = requests.get(challenger_url, headers = request_header).json()

# grand master

grandmaster_url = base_url + 'league/v1/grandmaster'

grandmaster = requests.get(grandmaster_url, headers = request_header).json()

# master

master_url = base_url + 'league/v1/master'

master = requests.get(master_url, headers = request_header).json()

In [4]:
# 가져온 챌린저 데이터 프레임 조회 (마스터 랭크에 속한 유저 조회)
pd.DataFrame(challenger).head(3)

# 가져온 그랜드마스터 데이터 프레임 조회 (그랜드마스터 랭크에 속한 유저 조회)
pd.DataFrame(grandmaster).head(3)

# 가져온 마스터 데이터 프레임 조회 (마스터 랭크에 속한 유저 조회)
pd.DataFrame(master).head(3)

,tier,leagueId,queue,name,entries
0,MASTER,c9353fab-a304-3562-bdf5-e7375f43e1cd,RANKED_TFT,Karthus's Executioners,"{'summonerId': 'ejplkuJkm0edS0SuGLEZH5iXGdTxTQYxeVhyPsvE0V6tlQ', 'summonerName': '탑 렝가', 'leaguePoints': 21, 'rank': 'I', 'wins': 129, 'losses': 114, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"
1,MASTER,c9353fab-a304-3562-bdf5-e7375f43e1cd,RANKED_TFT,Karthus's Executioners,"{'summonerId': 'Hj-TH9N_eHhJTKQkp3_juRujvrX-fxR7JgDyTNfvxjPw6z0', 'summonerName': '참예우', 'leaguePoints': 18, 'rank': 'I', 'wins': 90, 'losses': 79, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"
2,MASTER,c9353fab-a304-3562-bdf5-e7375f43e1cd,RANKED_TFT,Karthus's Executioners,"{'summonerId': 'lzxC4E-NrJZUymK-j-g2s3QnUGJjGe8bgS9JotZZ0RT5kIEjdQZ75xOBnA', 'summonerName': 'yqas', 'leaguePoints': 48, 'rank': 'I', 'wins': 75, 'losses': 64, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"


##### 추출한 랭크 구간별 유저 정보 데이터 프레임화

In [5]:
# challenger_df 에 저장
challenger_df = pd.DataFrame(challenger)

# grandmaster_df 에 저장
grandmaster_df = pd.DataFrame(grandmaster)

# master_df 에 저장
master_df = pd.DataFrame(master)

##### json Normalize, 필요없는 컬럼 drop 하는 함수 생성

In [6]:
def json_normalize_and_dropna(df):
    
    # entries 컬럼 확장  -  json 형식의 entries  컬럼 안에 중요 정보가 대부분 담겨있음
    entries_normalized = json_normalize(df['entries'])
    
    # 확장된 데이터프레임 병합
    df = pd.concat([df.drop(['entries'], axis=1), entries_normalized], axis=1)
    
    # 필요없는 컬럼 drop('queue', 'name')
    df = df.drop(['leagueId', 'queue', 'name', 'rank', 'veteran', 'inactive', 'freshBlood', 'hotStreak'], axis=1)
    
    return df

##### 챌린저, 그랜드 마스터, 마스터 유저 정보 데이터 프레임에 함수 적용

In [7]:
# challenger df 결측치 제거 및 json normalize
challenger_df = json_normalize_and_dropna(challenger_df) 

# grandmaster df 결측치 제거 및 json normalize
grandmaster_df = json_normalize_and_dropna(grandmaster_df)

# grandmaster df 결측치 제거 및 json normalize
master_df = json_normalize_and_dropna(master_df)

In [8]:
grandmaster_df.head(3)

,tier,summonerId,summonerName,leaguePoints,wins,losses
0,GRANDMASTER,vVSpixPqyFCYkhJjuoLYII3hKeByZt2BddiCzcJ3wXvFWg,Molcar,219,79,39
1,GRANDMASTER,uLnK1zSBzDkwR3HFoz195RlCpHq8TNVbVJBh7p_RC_JqBOwSGtd01KvOQg,WAWb,0,80,66
2,GRANDMASTER,R99sYTl70oL4j6iBxmdIU0eCRo-HTgOuAhsetCQ5R8dzcXsjWCb3YqT6-w,COPY C,361,64,27


##### 소환사 이름 데이터 프레임을 넣으면 닉네임에 대응하는 puuid 를 붙여서 반환하는 함수 생성

##### 먼저 닉네임을 넣으면 대응하는 puuid 를 반환하는 함수 생성

In [9]:
def get_puuid(summonerName):
    # 닉네임을 넣어줄 url 저장
    summonerName_url = base_url + "summoner/v1/summoners/by-name/" + summonerName

    # 닉네임을 넣었을 때, 반환값 저장
    have_puuid_table = requests.get(summonerName_url, headers = request_header).json()

    # puuid 추출
    return have_puuid_table['puuid']

In [10]:
# 함수 작동 테스트
get_puuid('COPY O')

't_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw'

##### get_puuid 함수를 이용해서 대응하는 puuid 를 새로운 컬럼으로 추가하여 반환하는 함수 생성

In [11]:
def get_puuid_column(df):
    # 새로운 컬럼을 추가할 리스트 생성
    puuid_list = []

    # 데이터프레임에서 소환사 이름 컬럼 찾기
    summonerName_column = None
    for column in df.columns:
        if 'summonerName' in column:
            summonerName_column = column
            break
    
    if summonerName_column is None:
        print("No column containing summoner names found in the dataframe.")
        return df

    # 각 소환사 이름에 대해 puuid 가져오기
    for summonerName in df[summonerName_column]:
        try:
            # 소환사 이름으로 puuid 가져오기
            puuid = get_puuid(summonerName)
            puuid_list.append(puuid)
        except Exception as e:
            print(f"Error getting PUUID for summoner '{summonerName}': {str(e)}")
            puuid_list.append(None)  # 에러가 발생하면 None을 추가

    # 새로운 컬럼을 데이터프레임에 추가
    df['puuid'] = puuid_list

    return df

##### get_puuid_column(df) 함수 챌,그마,마 테이블에 적용

In [12]:
# challanger
get_puuid_column(challenger_df) 

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,CHALLENGER,UF2IeZC87PAxxS0F7xg6VQNADum-YqoBLfYCOAb0LdmQL7WIfnII-3Cm1Q,kscv05,525,105,57,mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ
1,CHALLENGER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,764,98,45,t_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw
2,CHALLENGER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,586,103,65,Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ
3,CHALLENGER,0sgFw0oa2pxGV5rZR3Fi96VY8t6500V3t3TEso8XyyKmqHhQ4xURXo5oKA,lllIlIllIlIlIllI,646,76,19,M6OtnT2Ad4fhM9Sj1fh1odtr0p4oO6rqxc6qtW_lPlrkX8wfqUm-LMYHb_SRi56T8-X0npFDIZj9GQ
4,CHALLENGER,xszNgAg-6vRPAVyxqeu3L9oAeiFW8OnunPj6wbAbwAq_wVw,존잘인천피스트,529,101,78,7ITuO8wSX3-KnWipQVyAnnx7y75RPmOOU2TVy42-fSya7UXdwHfioyIG7jn6owP-tkUzrPfOU7Q4RQ
5,CHALLENGER,yu3ysnF3p8f8yFitlLc1vrg8akquavQKXWWtFgzkT3syIRI,guiId,459,86,58,Td6_AZDCl2E_9hk60D8dzx0Ylm9SmQS79vyMP1RDxAaUoRqSP-uJrCIc2IVnSCrEDrjlMc3qOgyj5g
6,CHALLENGER,wzBjtuG_WMAjyCEBAs3L8qBOf-bkIM1WVU_HM2VUmTeWoeE,The Smile,642,83,31,T0jmK1y7NP42XF5mrGZz7ayoGGik0BXMpFaLXUW1w8ouLcPyhD4Yw0IXcNLCdamegk2CFgFQbJ9mBQ
7,CHALLENGER,W1eFDHJMN6FqZNL6PAP2czGWRjDbaa2YsB4Zqu53PHDp2Y89ZtecsLN1Wg,COPY P,680,100,54,iCcuYtTqqr1BKUjBwIDfE1FWhLj4Nog495kIOrxNqEfWq6iboiWi8epQ443aAIBGsy4ej3pdRyrfuQ


In [13]:
# 챌린저 실행 후 2분 뒤 실행
# grandmaster
get_puuid_column(grandmaster_df)

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,GRANDMASTER,vVSpixPqyFCYkhJjuoLYII3hKeByZt2BddiCzcJ3wXvFWg,Molcar,219,79,39,rqc-kP7lHERzim8AkpRdX_BQ7HP6cWbEGnsf_cIX63f7OUBC7hLRYWFnvItxM3_fRg7_HweVUlqm1w
1,GRANDMASTER,uLnK1zSBzDkwR3HFoz195RlCpHq8TNVbVJBh7p_RC_JqBOwSGtd01KvOQg,WAWb,0,80,66,xzvZ2Dz95XrGaT8ohOUMCFsiRR45X8wYTJJ-eECnYn4Iw8akLlP1kIIOY6mHaT0i5nUsvtbw1JSJTw
2,GRANDMASTER,R99sYTl70oL4j6iBxmdIU0eCRo-HTgOuAhsetCQ5R8dzcXsjWCb3YqT6-w,COPY C,361,64,27,sV7rWaPHoFjEfvl08t9S7g6GXaExIb_mFB5zJnjxrnpyz194sQ9cPmMXEmDMKxih5iWucZRngigxPA
3,GRANDMASTER,pNUDUaztjVI1G7Jzoh4aVDMGr_8nDd0sMVeuWOvzwNMC6vB5,MessiOfTFT,514,110,71,4_-cZaXUQM6l7UQGnB69F9007HRO3dyTdKP-7_u0utPCg6YeZCBKMOZvoLFUdNyDa78Y7RBVqBXcGQ
4,GRANDMASTER,KKtx2l6oWWPWxU6Pi5tm0trZQI6yFkVbHjyMALGori8cG24DSiXPdy5gug,천연만화,512,91,47,lskGGafPGWafTOrBtnSVRCyYdggWRybYvU_B0r65qQvnGCU2C4gmZmt9rMDAnruQuoloPyWnpeBiqQ
5,GRANDMASTER,lRy2SJbcvIf0f1v5QN5BknGQviN9MG4JQAix8jfijl1wr4g,발 명,339,88,47,oHvcdDVV-nVXEBDN5ur13eTS5xqZetOML9jAmYGQ0TJ7cS7gSm32jlKylYgNRNJQsqnVcoVacydsRg
6,GRANDMASTER,VVpwkqbuN3VVwG378diAEHea6jFrq1ADYBQAEstZizthf7k,이렇게 좋은 날,297,120,77,rx1nmyxEddDyQujL2nvjfiHV_LDcBEo-zyf16l3BFnXQExOMoB6tNss-WGlc_GRgcac2tg1GhDgBhg
7,GRANDMASTER,FrXrN1Pv2I33YcX3QBw3-KYcYqfr--UrnYZnE1uv_ctGH_k,밀짚모자 쌍옆,581,105,40,IgNThWgULt0X9wkf1O_V80FmpnuBx0xvLonX-82LwbzEIXT8DtxwJArPXvpcDhf0rwIwrsXo_lNO5g
8,GRANDMASTER,NIkaHhNShS6_POtI2cpVIADO53U0H2SVqwZd3X89Wn0Apqk,머저리 깅중밍,388,93,54,tvrulWGLrA7Au-z6OZlh_jAe60ij5TsBuKKdl7tCoUFIrOrdDtnglUgm70IWVSsvHLPgBbi8LXAsLQ
9,GRANDMASTER,SGNQQczl7q3V6optBsoGdC1Ouk_emCoJoi87FS6k3FOXFSk,빵빵한가아슴,395,109,72,94SMpcSJOkUEO1XLoB0a52NC089M82djDPobBmNbTgELEa7DFDrp6LGXsRk5JkzHt_0PDk4sKEs9Jg


In [14]:
# 그랜드 마스터 실행 후 2분 뒤 실행
# master
get_puuid_column(master_df)

Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '': 'puuid'
Error getting PUUID for summoner '르꼬르뷔지어l': 'puuid'
Error getting PUUID for summoner '덕소나몽이': 'puuid'
Error getting PUUID for summoner '강한해린': 'puuid'
Error getting PUUID for summoner '송제송': 'puuid'
Error getting PUUID for summoner '얘 그걸로 되겠니': 'puuid'
Error getting PUUID for summoner '그저게임가지고왜': 'puuid'
Error getting PUUID for summoner '개구리는개굴': 'puuid'
Error getting PUUID for summoner '뜨또의실험실': 'puuid'
Error g

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,MASTER,ejplkuJkm0edS0SuGLEZH5iXGdTxTQYxeVhyPsvE0V6tlQ,탑 렝가,21,129,114,BLKLr0C6Px7qhf5vNe5jyzYIK2h_QRO2T69teeCtMMU8rGEAV3ZK2hpKlZemnjl1Q6Ti5XnY5G1lRw
1,MASTER,Hj-TH9N_eHhJTKQkp3_juRujvrX-fxR7JgDyTNfvxjPw6z0,참예우,18,90,79,nDrKVvfIrYbU0nDSaTozznVM1PAcgJXhq4kKHi68l7BTk-8CWSelMwdAjF_84ZgYHDQ40uFH_Cj5Zw
2,MASTER,lzxC4E-NrJZUymK-j-g2s3QnUGJjGe8bgS9JotZZ0RT5kIEjdQZ75xOBnA,yqas,48,75,64,nRyOO54JqNYwFDYO7AtKcKZtbAWaNRyz1yIA0xRsJoRIF5faHM3uPGr59o9O9velaaOBiEMYR51CXw
3,MASTER,gv4yMS-QuDyRtLNZG7L0f3k4fUS8KR5Kek-XV9Ns_76R7g,그거그거그거그거,2,65,44,v32J8YX965j9DJ2sC4cBGCVnLVhFIjmg0B8BNtibafkLT47KfggTMZJqOf5LDezcOWzTXX-jrE8n6w
4,MASTER,CfZG_I40rUdET1vdIl5rdPbb1_dArmLG5B65HG9iE1XY2rM,투쁠러스원,0,61,40,L2klDTcJGKV8gTx2DBr14za2xaChh2M6V8hXSWumR3aK4O8NfPYyCJEjRUbtw5SnIXFRVdjRMbM_YQ
...,...,...,...,...,...,...,...
129,MASTER,b_c6A7LuTkK3FT5U4rbakabsPF8ARG2DdJg1u_z8HcuO50k,EATPEPSI,1,128,126,dmMPPn5VRimw0dLGh2_885MjOe0nWfygW_nFei_g4wrpv9mBC0wPMyM-0JIvUwAT-B19Be_Xex9bFA
130,MASTER,1G9ZilkyrgxWPWMUaq_RtmJE5q7f2BijO3_3hMsIm2RB69s,T1 골픽이,84,80,59,_tFZFQDxr3ipe-amA8IMZiUugmCq6rhTN9YrLPthGkJSnh1TbAB9j1v4_pUGvH7UWmwnNM_jUtDhAw
131,MASTER,_nVrK0nlNpn10YBdoeSYm10lia_NQNylppoeZ-c5ZnXIwuFzxOaFAzVdjw,,217,58,19,None
132,MASTER,mhHHuZ8PK2gYk7PpyrhKSLt0EwrRBHe1rk5DT3VDBhmiZPPvpqziAgQZtw,무진스 협회장,28,71,43,Pb2S0Ec6nyYB5qJCRf9eUe_9aGjKtPwkOVADulKwdpD_rlOGB6I1FCcPGaiPYFDrzT6A9rVPNlpV2Q


In [15]:
# 확인
grandmaster_df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,GRANDMASTER,vVSpixPqyFCYkhJjuoLYII3hKeByZt2BddiCzcJ3wXvFWg,Molcar,219,79,39,rqc-kP7lHERzim8AkpRdX_BQ7HP6cWbEGnsf_cIX63f7OUBC7hLRYWFnvItxM3_fRg7_HweVUlqm1w
1,GRANDMASTER,uLnK1zSBzDkwR3HFoz195RlCpHq8TNVbVJBh7p_RC_JqBOwSGtd01KvOQg,WAWb,0,80,66,xzvZ2Dz95XrGaT8ohOUMCFsiRR45X8wYTJJ-eECnYn4Iw8akLlP1kIIOY6mHaT0i5nUsvtbw1JSJTw
2,GRANDMASTER,R99sYTl70oL4j6iBxmdIU0eCRo-HTgOuAhsetCQ5R8dzcXsjWCb3YqT6-w,COPY C,361,64,27,sV7rWaPHoFjEfvl08t9S7g6GXaExIb_mFB5zJnjxrnpyz194sQ9cPmMXEmDMKxih5iWucZRngigxPA
3,GRANDMASTER,pNUDUaztjVI1G7Jzoh4aVDMGr_8nDd0sMVeuWOvzwNMC6vB5,MessiOfTFT,514,110,71,4_-cZaXUQM6l7UQGnB69F9007HRO3dyTdKP-7_u0utPCg6YeZCBKMOZvoLFUdNyDa78Y7RBVqBXcGQ
4,GRANDMASTER,KKtx2l6oWWPWxU6Pi5tm0trZQI6yFkVbHjyMALGori8cG24DSiXPdy5gug,천연만화,512,91,47,lskGGafPGWafTOrBtnSVRCyYdggWRybYvU_B0r65qQvnGCU2C4gmZmt9rMDAnruQuoloPyWnpeBiqQ


##### 만들 테이블들(챌린저, 그마, 마스터) 합쳐서 하나의 테이블로 만들기

In [16]:
# 챌린저 티어 유저 생기면 아래에 있는 쿼리 주석 제거하고 사용
combined_df = pd.concat([challenger_df, grandmaster_df, master_df], ignore_index=True)

combined_df

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,CHALLENGER,UF2IeZC87PAxxS0F7xg6VQNADum-YqoBLfYCOAb0LdmQL7WIfnII-3Cm1Q,kscv05,525,105,57,mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ
1,CHALLENGER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,764,98,45,t_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw
2,CHALLENGER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,586,103,65,Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ
3,CHALLENGER,0sgFw0oa2pxGV5rZR3Fi96VY8t6500V3t3TEso8XyyKmqHhQ4xURXo5oKA,lllIlIllIlIlIllI,646,76,19,M6OtnT2Ad4fhM9Sj1fh1odtr0p4oO6rqxc6qtW_lPlrkX8wfqUm-LMYHb_SRi56T8-X0npFDIZj9GQ
4,CHALLENGER,xszNgAg-6vRPAVyxqeu3L9oAeiFW8OnunPj6wbAbwAq_wVw,존잘인천피스트,529,101,78,7ITuO8wSX3-KnWipQVyAnnx7y75RPmOOU2TVy42-fSya7UXdwHfioyIG7jn6owP-tkUzrPfOU7Q4RQ
...,...,...,...,...,...,...,...
161,MASTER,b_c6A7LuTkK3FT5U4rbakabsPF8ARG2DdJg1u_z8HcuO50k,EATPEPSI,1,128,126,dmMPPn5VRimw0dLGh2_885MjOe0nWfygW_nFei_g4wrpv9mBC0wPMyM-0JIvUwAT-B19Be_Xex9bFA
162,MASTER,1G9ZilkyrgxWPWMUaq_RtmJE5q7f2BijO3_3hMsIm2RB69s,T1 골픽이,84,80,59,_tFZFQDxr3ipe-amA8IMZiUugmCq6rhTN9YrLPthGkJSnh1TbAB9j1v4_pUGvH7UWmwnNM_jUtDhAw
163,MASTER,_nVrK0nlNpn10YBdoeSYm10lia_NQNylppoeZ-c5ZnXIwuFzxOaFAzVdjw,,217,58,19,None
164,MASTER,mhHHuZ8PK2gYk7PpyrhKSLt0EwrRBHe1rk5DT3VDBhmiZPPvpqziAgQZtw,무진스 협회장,28,71,43,Pb2S0Ec6nyYB5qJCRf9eUe_9aGjKtPwkOVADulKwdpD_rlOGB6I1FCcPGaiPYFDrzT6A9rVPNlpV2Q


##### 결측치 제거 - 합친 데이터 테이블에서 puuid 가 null 인 행 제거

In [17]:
combined_df = combined_df.dropna(subset=['puuid'])

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138 entries, 0 to 165
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          138 non-null    object
 1   summonerId    138 non-null    object
 2   summonerName  138 non-null    object
 3   leaguePoints  138 non-null    int64 
 4   wins          138 non-null    int64 
 5   losses        138 non-null    int64 
 6   puuid         138 non-null    object
dtypes: int64(3), object(4)
memory usage: 8.6+ KB


##### 전처리 완료한 테이블 csv 로 저장

In [18]:
combined_df.to_csv('ranker_name_puuid.csv', index=False)

In [19]:
# 저장된 테이블 확인
pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/ranker_name_puuid.csv')

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,CHALLENGER,UF2IeZC87PAxxS0F7xg6VQNADum-YqoBLfYCOAb0LdmQL7WIfnII-3Cm1Q,kscv05,525,105,57,mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ
1,CHALLENGER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,764,98,45,t_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw
2,CHALLENGER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,586,103,65,Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ
3,CHALLENGER,0sgFw0oa2pxGV5rZR3Fi96VY8t6500V3t3TEso8XyyKmqHhQ4xURXo5oKA,lllIlIllIlIlIllI,646,76,19,M6OtnT2Ad4fhM9Sj1fh1odtr0p4oO6rqxc6qtW_lPlrkX8wfqUm-LMYHb_SRi56T8-X0npFDIZj9GQ
4,CHALLENGER,xszNgAg-6vRPAVyxqeu3L9oAeiFW8OnunPj6wbAbwAq_wVw,존잘인천피스트,529,101,78,7ITuO8wSX3-KnWipQVyAnnx7y75RPmOOU2TVy42-fSya7UXdwHfioyIG7jn6owP-tkUzrPfOU7Q4RQ
...,...,...,...,...,...,...,...
133,MASTER,1fyiJJRE0IVRhbnEOwKXOElDvbkgEU-iSxcVtZscP-KFcOfPvnqwb2GpBw,카지노딜도,10,56,32,r4_G_3u5p7OftAqV5CbnrWO-6WXWcEZVBhylFGVB0HZNjt-cluwVmV-cLYhFK_2m04EOz2-BpZ6ZUA
134,MASTER,b_c6A7LuTkK3FT5U4rbakabsPF8ARG2DdJg1u_z8HcuO50k,EATPEPSI,1,128,126,dmMPPn5VRimw0dLGh2_885MjOe0nWfygW_nFei_g4wrpv9mBC0wPMyM-0JIvUwAT-B19Be_Xex9bFA
135,MASTER,1G9ZilkyrgxWPWMUaq_RtmJE5q7f2BijO3_3hMsIm2RB69s,T1 골픽이,84,80,59,_tFZFQDxr3ipe-amA8IMZiUugmCq6rhTN9YrLPthGkJSnh1TbAB9j1v4_pUGvH7UWmwnNM_jUtDhAw
136,MASTER,mhHHuZ8PK2gYk7PpyrhKSLt0EwrRBHe1rk5DT3VDBhmiZPPvpqziAgQZtw,무진스 협회장,28,71,43,Pb2S0Ec6nyYB5qJCRf9eUe_9aGjKtPwkOVADulKwdpD_rlOGB6I1FCcPGaiPYFDrzT6A9rVPNlpV2Q
